In [16]:
import numpy as np

covs_all = np.load('../data/simulation2/covs_all.npy')
cov = np.load('../data/simulation2/cov_pop.npy')

In [2]:
num_cells = 10
len_series = np.array([68, 116, 216, 216,  94, 216, 216, 190, 216, 216],dtype=np.int64)

In [3]:
ns02 = np.zeros((21,11),dtype=np.int64)

In [4]:
list0 = [0,3,4,6,7,9,10,11,12,13,15,18,19,20]
list1 = [2,3,7,8,9,12,14,15,16,17,18,20,24,25,26,28,31,32,33]
list2 = [2,4,5,6,7,8,9,10]

In [5]:
for i in list0:
    for j in list2:
        #print(i,j)
        eigvals = np.zeros((num_cells,2))
        sqrt_eigvals = np.zeros((num_cells,2))
        eigvecs = np.zeros((num_cells,2,2))
        for c in range(num_cells):
            #mat = np.array([[S00[c],covs[c,0,1],covs[c,0,2],covs[c,0,6]],[covs[c,1,0],covs[c,1,1],covs[c,1,2],covs[c,1,6]],[covs[c,2,0],covs[c,2,1],covs[c,2,2],covs[c,2,6]],[covs[c,6,0],covs[c,6,1],covs[c,6,2],covs[c,6,6]]])
            #eigvals[c], eigvecs[c] = np.linalg.eigh(mat)
            eigvals[c], eigvecs[c] = np.linalg.eigh(covs_all[np.ix_([c],[i+4,j+59],[i+4,j+59])])
            sqrt_eigvals[c] = np.sqrt(eigvals[c])

        num_trials = 5000000
        vals = np.zeros((num_trials,num_cells,2))
        for tr in range(num_trials):
            #if tr%10000==0: print(tr)
            for c in range(num_cells):
                vals[tr,c] = eigvecs[c]@(sqrt_eigvals[c]*np.random.normal(size=(2,)))
        
        means_sim = np.zeros((num_trials,2))
        means_sim = np.sum(vals*len_series[np.newaxis,:,np.newaxis], axis=1)/np.sum(len_series[:])
        
        covs_sim = np.zeros((num_trials,2,2))
        for k in range(2):
            for l in range(2):
                covs_sim[:,k,l] = np.sum(len_series[np.newaxis,:]*(vals[:,:,k]-means_sim[:,np.newaxis,k])*(vals[:,:,l]-means_sim[:,np.newaxis,l]), axis=1)/np.sum(len_series[:])
        covs_diff = cov[np.ix_([i+4,j+59],[i+4,j+59])][np.newaxis,:,:] - covs_sim
        
        #scale_mat_inv = np.linalg.inv(scale_mat[np.ix_([i,j],[i,j])])
        #mat_sim = np.matmul(covs_diff, scale_mat_inv)
        a = covs_diff[:,0,0]
        b = covs_diff[:,0,1]
        c = covs_diff[:,1,0]
        d = covs_diff[:,1,1]
        eigs_sim = (a+d-np.sqrt((a-d)**2+4*b*c))/2
        ns02[i,j] = np.sum(np.where(eigs_sim<=0,1,0))
        print(i,j,ns02[i,j])

0 2 49835
0 4 1260
0 5 19149
0 6 3324
0 7 3101
0 8 1033
0 9 5258
0 10 2153
3 2 57950
3 4 1904
3 5 581693
3 6 11720
3 7 5038
3 8 2133
3 9 640730
3 10 7206
4 2 0
4 4 0
4 5 904
4 6 0
4 7 149
4 8 0
4 9 36
4 10 0
6 2 0
6 4 41357
6 5 60143
6 6 11878
6 7 3244613
6 8 3435
6 9 5078
6 10 141
7 2 6212
7 4 627241
7 5 9510
7 6 51735
7 7 690779
7 8 538145
7 9 4443
7 10 23673
9 2 12
9 4 96
9 5 5260
9 6 991
9 7 750444
9 8 0
9 9 23
9 10 0
10 2 1
10 4 9
10 5 381
10 6 0
10 7 202
10 8 0
10 9 0
10 10 0
11 2 16
11 4 0
11 5 8
11 6 0
11 7 211
11 8 0
11 9 0
11 10 0
12 2 370
12 4 839
12 5 72656
12 6 10717
12 7 4788326
12 8 131
12 9 13437
12 10 52
13 2 3
13 4 13774
13 5 642970
13 6 63126
13 7 1880117
13 8 6250
13 9 809739
13 10 33588
15 2 10
15 4 133381
15 5 77987
15 6 2130
15 7 2875852
15 8 54460
15 9 8841
15 10 2
18 2 13
18 4 8699
18 5 1735
18 6 464
18 7 23914
18 8 92
18 9 2
18 10 3
19 2 2954
19 4 362172
19 5 7028
19 6 2006
19 7 19840
19 8 399941
19 9 3065
19 10 1939
20 2 2443
20 4 13390
20 5 108
20 6 14
20 7 

In [6]:
with open('../data/simulation2/ns02.npy','wb') as f:
    np.save(f, ns02)

In [8]:
ns02 = np.load('../data/simulation2/ns02.npy')

In [9]:
for i in [0]:
    for j in list2:
        #print(i,j)
        eigvals = np.zeros((num_cells,2))
        sqrt_eigvals = np.zeros((num_cells,2))
        eigvecs = np.zeros((num_cells,2,2))
        for c in range(num_cells):
            #mat = np.array([[S00[c],covs[c,0,1],covs[c,0,2],covs[c,0,6]],[covs[c,1,0],covs[c,1,1],covs[c,1,2],covs[c,1,6]],[covs[c,2,0],covs[c,2,1],covs[c,2,2],covs[c,2,6]],[covs[c,6,0],covs[c,6,1],covs[c,6,2],covs[c,6,6]]])
            #eigvals[c], eigvecs[c] = np.linalg.eigh(mat)
            eigvals[c], eigvecs[c] = np.linalg.eigh(covs_all[np.ix_([c],[i+4,j+59],[i+4,j+59])])
            sqrt_eigvals[c] = np.sqrt(eigvals[c])

        num_trials = 5000000
        vals = np.zeros((num_trials,num_cells,2))
        for tr in range(num_trials):
            #if tr%10000==0: print(tr)
            for c in range(num_cells):
                vals[tr,c] = eigvecs[c]@(sqrt_eigvals[c]*np.random.normal(size=(2,)))
        
        means_sim = np.zeros((num_trials,2))
        means_sim = np.sum(vals*len_series[np.newaxis,:,np.newaxis], axis=1)/np.sum(len_series[:])
        
        covs_sim = np.zeros((num_trials,2,2))
        for k in range(2):
            for l in range(2):
                covs_sim[:,k,l] = np.sum(len_series[np.newaxis,:]*(vals[:,:,k]-means_sim[:,np.newaxis,k])*(vals[:,:,l]-means_sim[:,np.newaxis,l]), axis=1)/np.sum(len_series[:])
        covs_diff = cov[np.ix_([i+4,j+59],[i+4,j+59])][np.newaxis,:,:] - covs_sim
        
        #scale_mat_inv = np.linalg.inv(scale_mat[np.ix_([i,j],[i,j])])
        #mat_sim = np.matmul(covs_diff, scale_mat_inv)
        a = covs_diff[:,0,0]
        b = covs_diff[:,0,1]
        c = covs_diff[:,1,0]
        d = covs_diff[:,1,1]
        eigs_sim = (a+d-np.sqrt((a-d)**2+4*b*c))/2
        ns02[i,j] = np.sum(np.where(eigs_sim<=0,1,0))
        print(i,j,ns02[i,j])

0 2 48520
0 4 1218
0 5 18465
0 6 3198
0 7 2863
0 8 1099
0 9 5030
0 10 2179


In [10]:
with open('../data/simulation2/ns02.npy','wb') as f:
    np.save(f, ns02)

In [17]:
ns02 = np.load('../data/simulation2/ns02.npy')

In [18]:
for i in [13]:
    for j in list2:
        #print(i,j)
        eigvals = np.zeros((num_cells,2))
        sqrt_eigvals = np.zeros((num_cells,2))
        eigvecs = np.zeros((num_cells,2,2))
        for c in range(num_cells):
            #mat = np.array([[S00[c],covs[c,0,1],covs[c,0,2],covs[c,0,6]],[covs[c,1,0],covs[c,1,1],covs[c,1,2],covs[c,1,6]],[covs[c,2,0],covs[c,2,1],covs[c,2,2],covs[c,2,6]],[covs[c,6,0],covs[c,6,1],covs[c,6,2],covs[c,6,6]]])
            #eigvals[c], eigvecs[c] = np.linalg.eigh(mat)
            eigvals[c], eigvecs[c] = np.linalg.eigh(covs_all[np.ix_([c],[i+4,j+59],[i+4,j+59])])
            sqrt_eigvals[c] = np.sqrt(eigvals[c])

        num_trials = 5000000
        vals = np.zeros((num_trials,num_cells,2))
        for tr in range(num_trials):
            #if tr%10000==0: print(tr)
            for c in range(num_cells):
                vals[tr,c] = eigvecs[c]@(sqrt_eigvals[c]*np.random.normal(size=(2,)))
        
        means_sim = np.zeros((num_trials,2))
        means_sim = np.sum(vals*len_series[np.newaxis,:,np.newaxis], axis=1)/np.sum(len_series[:])
        
        covs_sim = np.zeros((num_trials,2,2))
        for k in range(2):
            for l in range(2):
                covs_sim[:,k,l] = np.sum(len_series[np.newaxis,:]*(vals[:,:,k]-means_sim[:,np.newaxis,k])*(vals[:,:,l]-means_sim[:,np.newaxis,l]), axis=1)/np.sum(len_series[:])
        covs_diff = cov[np.ix_([i+4,j+59],[i+4,j+59])][np.newaxis,:,:] - covs_sim
        
        #scale_mat_inv = np.linalg.inv(scale_mat[np.ix_([i,j],[i,j])])
        #mat_sim = np.matmul(covs_diff, scale_mat_inv)
        a = covs_diff[:,0,0]
        b = covs_diff[:,0,1]
        c = covs_diff[:,1,0]
        d = covs_diff[:,1,1]
        eigs_sim = (a+d-np.sqrt((a-d)**2+4*b*c))/2
        ns02[i,j] = np.sum(np.where(eigs_sim<=0,1,0))
        print(i,j,ns02[i,j])

13 2 3
13 4 15047
13 5 657786
13 6 68440
13 7 1901898
13 8 6850
13 9 830525
13 10 36273


In [19]:
with open('../data/simulation2/ns02.npy','wb') as f:
    np.save(f, ns02)